## LLMChain

## Imports

In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.text_splitter import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain

## Loading Environment Variables

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

## Instantiate Model

In [3]:
chat_model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",
                        max_tokens=2000,
                        temperature=0.3)

## Prompt Template

In [4]:
human_prompt = HumanMessagePromptTemplate.from_template(template='Make up a funny name for a company that makes: {product}')

In [5]:
chat_prompt_template = ChatPromptTemplate.from_messages([human_prompt])

## Running the Chain

In [6]:
chain = chat_prompt_template | chat_model

In [7]:
response = chain.invoke({"product": "Computers"})

In [8]:
print(response.content)

ByteMe Computers!


## Simple Sequential Chain

In [9]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

# Initialize your Chat model
chat_model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",
                        max_tokens=2000,
                        temperature=0.3)

# Define the prompt templates
template1 = "Give me a simple bullet point outline for a blog post on {topic}"
template2 = "Write a blog post using this outline:\n\n{outline}"

# Create ChatPromptTemplates from the templates
prompt1 = ChatPromptTemplate.from_template(template1)
prompt2 = ChatPromptTemplate.from_template(template2)

# Define the chains using LCEL (LangChain Expression Language)
chain = ({"outline": prompt1 | chat_model | StrOutputParser()}
         | prompt2
         | chat_model
         | StrOutputParser()
         )

In [10]:
result = chain.invoke({"topic": "Artificial Intelligence"})

In [11]:
# No need to extract from result.content because of StrOutputParser()
print(result)

# Understanding Artificial Intelligence: A Comprehensive Guide

## Introduction

Artificial Intelligence (AI) is a branch of computer science that aims to create machines capable of performing tasks that typically require human intelligence. This includes learning, reasoning, problem-solving, perception, and language understanding. In today's rapidly evolving technological landscape, AI is not just a buzzword; it is a transformative force that is reshaping industries, enhancing daily life, and influencing the way we interact with the world. In this blog post, we will delve into the fundamentals of AI, explore its workings, applications, benefits, challenges, and the future it holds.

## 1. What is Artificial Intelligence?

At its core, Artificial Intelligence refers to the simulation of human intelligence processes by machines, especially computer systems. Key concepts include machine learning, where systems learn from data, and neural networks, which are designed to mimic the human br

## Sequential Chain

### Deprecated syntax

In [12]:
template1 = "Give a summary of this employee's performance review: \n{review}"
prompt1 = ChatPromptTemplate.from_template(template1)
chain1 = LLMChain(llm=chat_model, prompt=prompt1, output_key='review_summary')

template2 = "Identify key employee weaknesses in this review summary: \n{review}"
prompt2 = ChatPromptTemplate.from_template(template2)
chain2 = LLMChain(llm=chat_model, prompt=prompt2, output_key='weaknesses')

template3 = "Create a personalized plan to help and fix these weaknesses: \n{review}"
prompt3 = ChatPromptTemplate.from_template(template3)
chain3 = LLMChain(llm=chat_model, prompt=prompt3, output_key='final_plan')

C:\Users\JimYi\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [13]:
seq_chain = SequentialChain(chains=[chain1, chain2, chain3],
                            input_variables=['review'],
                            output_variables=['review_summary', 'weaknesses', 'final_plan'],
                            verbose=True)

results = seq_chain.invoke("The employee, Joe, is highly motivated. He works hard and write good python codes. However, he struggles with time management and prioritization.")



> Entering new SequentialChain chain...

> Finished chain.


In [14]:
print(results['final_plan'])

To help Joe improve his time management and prioritization skills while leveraging his motivation and coding abilities, we can create a personalized plan that includes specific strategies, tools, and practices. Here’s a structured approach:

### Personalized Development Plan for Joe

#### 1. **Assessment and Goal Setting**
   - **Self-Assessment:** Have Joe reflect on his current time management practices. Ask him to identify specific situations where he struggles with prioritization.
   - **SMART Goals:** Set specific, measurable, achievable, relevant, and time-bound goals. For example:
     - "By the end of the month, I will prioritize my tasks daily and reduce my project completion time by 20%."

#### 2. **Time Management Techniques**
   - **Pomodoro Technique:** Encourage Joe to work in focused bursts (25 minutes of work followed by a 5-minute break). This can help him maintain focus and reduce burnout.
   - **Time Blocking:** Help Joe allocate specific blocks of time in his calend

### New syntax

In [16]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.runnable import RunnableParallel

# Initialize your model (replace with your actual model and API key)
chat_model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",
                        max_tokens=2000,
                        temperature=0.3)

# Define prompt templates clearly
template1 = "Give a summary of this employee's performance review:\n{review}"
template2 = "Identify key employee weaknesses in this review summary:\n{review_summary}"
template3 = "Create a personalized plan to help and fix these weaknesses:\n{weaknesses}"

# Create prompt objects
prompt1 = ChatPromptTemplate.from_template(template1)
prompt2 = ChatPromptTemplate.from_template(template2)
prompt3 = ChatPromptTemplate.from_template(template3)

# Define chains using LCEL (LangChain Expression Language)
chain1 = {"review_summary": prompt1 | chat_model | StrOutputParser()}
chain2 = {"weaknesses": prompt2 | chat_model | StrOutputParser()}
chain3 = {"final_plan": prompt3 | chat_model | StrOutputParser()}

In [17]:
# Combine chains sequentially
seq_chain = (RunnablePassthrough.assign(**chain1)
             | RunnablePassthrough.assign(**chain2)
             | RunnablePassthrough.assign(**chain3)
             )

In [18]:
# Invoke the chain with your input
results = seq_chain.invoke({
    "review": "The employee, Joe, is highly motivated. He works hard and writes good python codes. However, he struggles with time management and prioritization."
})

# Print results
print("Review Summary:\n", results["review_summary"])
print("\nWeaknesses:\n", results["weaknesses"])
print("\nPersonalized Improvement Plan:\n", results["final_plan"])

Review Summary:
 Joe is a highly motivated employee who demonstrates a strong work ethic and produces quality Python code. However, he faces challenges with time management and prioritization, which can impact his overall performance.

Weaknesses:
 The key employee weaknesses identified in the review summary are:

1. **Time Management**: Joe struggles to manage his time effectively, which can lead to delays or inefficiencies in his work.

2. **Prioritization**: He has difficulty prioritizing tasks, which may affect his ability to focus on the most important or urgent projects. 

These weaknesses can impact his overall performance despite his strong work ethic and coding skills.

Personalized Improvement Plan:
 To help Joe improve his time management and prioritization skills, we can create a personalized development plan that includes specific strategies, tools, and actions he can take. This plan will focus on building habits and skills that will enhance his productivity and effectiven